<a href="https://colab.research.google.com/github/AlifiyulAkyun/SparkMLLib/blob/main/spark_mllib4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama : Alifiyul Akyun
* No : 04
* Kelas : TI3B
* NIM : 2041720036

In [1]:
# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Install PySpark
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=dbd50ce189c6d8c0bf8c8d28586d235d8d86e6c3bba75bda27d4635721050279
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
# Import Library
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col


In [4]:
spark = SparkSession.builder.appName("Movie Lens").getOrCreate()

In [5]:
# Parse String Menjadi Objek Rating
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3]))


In [6]:
# Baca File
raw = spark.read.text("/content/drive/MyDrive/bigdata-master/spark/examples/mllib/ratings.dat").rdd.map(lambda x: x[0])
ratings = raw.map(parseRating).toDF(["userId", "movieId", "rating", "timestamp"])
ratings.show(5)


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|   5.0|978300760|
|     1|    661|   3.0|978302109|
|     1|    914|   3.0|978301968|
|     1|   3408|   4.0|978300275|
|     1|   2355|   5.0|978824291|
+------+-------+------+---------+
only showing top 5 rows



In [7]:
# Data Training 80% dan Test 20%
training, test = ratings.randomSplit([0.8, 0.2])

# Membuat Model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)
model.save("mymodel")

In [8]:
# Prediksi Data
predictions = model.transform(test)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]
print("Mean Squared Error:", mse)

predictions.show(10)

Mean Squared Error: nan
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    527|   5.0|978824195|  4.650986|
|     1|    608|   4.0|978301398| 4.3566937|
|     1|    745|   3.0|978824268| 3.4603179|
|     1|   1022|   5.0|978300055|  4.631183|
|     1|   1270|   5.0|978300055|  4.272282|
|     1|   1566|   4.0|978824330|   3.00128|
|     1|   1836|   5.0|978300172| 3.5982795|
|     1|   1962|   4.0|978301753| 3.8153105|
|     1|   2018|   4.0|978301777|  5.066011|
|     1|   3114|   4.0|978302174| 4.3160086|
+------+-------+------+---------+----------+
only showing top 10 rows



In [9]:
# Menyimpan Hasil Prediksi
predictions.write.format("csv").save("ml-predictions.csv")